# Github README Demo

In [ ]:
import requests
import pandas as pd

data_path = "./dataset/data/esol_iupac.csv"
raw_data = pd.read_csv(data_path)

def query2IUPAC(text):
  try:
    '''This function queries the one given molecule name and returns a SMILES string from the record'''
    #query the PubChem database
    r = requests.get('https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/' + text + '/property/IUPACName/JSON')
    data = r.json()
    smi = data["PropertyTable"]["Properties"][0]["IUPACName"]
    return smi
  except:
    return None

# raw_data["IUPAC"] = raw_data["SMILES"].map(lambda sml: query2IUPAC(sml))
raw_data = raw_data[["IUPAC", "measured log(solubility:mol/L)"]]
raw_data = raw_data.dropna()
raw_data[50:65]

### README example

In [ ]:
import bolift
import openai
from dotenv import load_dotenv
load_dotenv()

asktell = bolift.AskTellFewShotTopk()

asktell.tell("3-chloroaniline", -1.37)
asktell.tell("nitromethane", 0.26)
asktell.tell("1-bromobutane", -2.43)
asktell.tell("3-chlorophenol", -0.7)

yhat = asktell.predict("penta-1,4-diene	")
print(yhat.mean(), yhat.std())

pool_list = [
  "1,5-dimethylnaphthalene",
  "2-aminophenol",
  "1hexa-1,5-diene",
  "1,1,2,3,4,4-hexachlorobuta-1,3-diene"
]
pool=bolift.Pool(pool_list)
print(asktell.ask(pool))

asktell.tell("phenol", -0.5)
yhat = asktell.predict("penta-1,4-diene")
print(yhat.mean(), yhat.std())

# Regression experiments

In [ ]:
from dotenv import load_dotenv
load_dotenv()

import os
import sys

current_dir = os.getcwd()
parent_dir = os.path.join(current_dir,'../bolift')
sys.path.insert(0, parent_dir)

import bolift
from bolift.llm_model import GaussDist, DiscreteDist
import numpy as np
import json
import pandas as pd
from langchain.prompts.prompt import PromptTemplate
import itertools
import openai
import glob

np.random.seed(0)

## Utils

In [ ]:
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)
# @retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def run_ablation_experiment(asktell, train_data, test_data, system_message=""):
    if isinstance(asktell, bolift.AskTellGPR) or isinstance(asktell, bolift.AskTellFinetuning):
        i=-1 #Hack to pass the case we have len(train_data) == 1
        for i in range(len(train_data)-1):
            asktell.tell(train_data.iloc[i, 0], float(train_data.iloc[i, 1]), train=False)
        asktell.tell(train_data.iloc[i+1, 0], float(train_data.iloc[i+1, 1]), train=True)
    else:
        for i in range(len(train_data)):
            asktell.tell(train_data.iloc[i, 0], float(train_data.iloc[i, 1]))
    x    = []
    y    = []
    yhat = []
    for j in range(len(test_data)):
        x.append(test_data.iloc[j, 0])
        y.append(float(test_data.iloc[j, 1]))
        if isinstance(asktell, bolift.AskTellNearestNeighbor):
            yhat.append(asktell.predict(test_data.iloc[j, 0]))
        else:
            yhat.append(asktell.predict(test_data.iloc[j, 0], system_message=system_message))

    x_filter = [xi for xi, yhi in zip(x, yhat)]# if len(yhi.values) > 0]
    y_filter = [yi for yi, yhi in zip(y, yhat)]# if len(yhi.values) > 0]
    yhat_filter = [yhi for yi, yhi in zip(y, yhat)]# if len(yhi.values) > 0]
    return x_filter, y_filter, yhat_filter

def save_csv(filename, x, y, yhat, data, model, T, k, N, model_class, tokens):
    if not os.path.exists(filename):
        f = open(filename, "w")
        f.write("y;yhat;yprobs;data;model;Temperature;k_selected;N_train;model_class;n_tokens;x\n")
    else:
        f = open(filename, "a")

    for xi, yi, yhi in zip(x, y, yhat):
        # print(yi, yhi, data, model, T, k, N, model_class, tokens, xi)
        if isinstance(yhi, DiscreteDist):
            if len(yhi.values) > 0:
                for v,p in zip(yhi.values, yhi.probs):
                    f.write(f"{yi};{v};{p:.4f};{data};{model};{T};{k};{N};{model_class};{tokens};{xi}\n")
        if isinstance(yhi, GaussDist):
            # f.write(f"{yi};{yhi.mean()};{yhi.std():.4f};{data};{model};{T};{k};{N};{model_class};{tokens};{xi}\n")
            f.write(f"{yi};{yhi.mean()};{0};{data};{model};{T};{k};{N};{model_class};{tokens};{xi}\n")
    f.close()


In [ ]:
def get_dataset(data: str, N: int, split=0.8):
    match data:
        case "in-house":
            data_path = "./dataset/data/71023_BO_ready_pool.csv"
            raw_data = pd.read_csv(data_path)
            raw_data['Catalyst'] = raw_data['Prompt'].str.extract(r'(\b[A-Z][a-z]?:[A-Z][a-z]?:[A-Z][a-z]?\b)')
            unique_cat = raw_data['Catalyst'].unique()
            c = {c: 0.2+m*(5/len(unique_cat)) for m, c in enumerate(unique_cat)}
            raw_data['dummy_Completion'] = raw_data['Catalyst'].apply(lambda x: np.random.normal(c[x], 0.05))

            x_name = "Prompt"
            y_name = "dummy_Completion"
        case "ocm":
            data_path = "./dataset/data/12708_ocm_dataset.csv"
            raw_data = pd.read_csv(data_path, sep=";")
            raw_data = raw_data.sample(frac=1).reset_index(drop=True)
            x_name = "prompt"
            y_name = "completion"
        case "biasfree_ocm":
            data_path = "./dataset/data/bias_free_ocmdataset_p_comp.csv"
            raw_data = pd.read_csv(data_path, sep=",")
            raw_data = raw_data.sample(frac=1).reset_index(drop=True)
            x_name = "prompt"
            y_name = "completion"
        case "sol":
            data_path = "./dataset/data/esol_iupac.csv"
            raw_data = pd.read_csv(data_path)
            raw_data = raw_data.dropna()
            raw_data = raw_data[["IUPAC", "measured log(solubility:mol/L)"]]
            raw_data = raw_data.sample(frac=1).reset_index(drop=True)
            x_name = "IUPAC"
            y_name = "measured log(solubility:mol/L)"
        case "alloy":
            data_path = "./dataset/data/charge_transfer_dataset.csv"
            raw_data = pd.read_csv(data_path, sep=",")
            raw_data = raw_data[['prompt', 'completion']]
            raw_data = raw_data.sample(frac=1).reset_index(drop=True)
            x_name = "prompt"
            y_name = "completion"
        case _:
            raise ValueError("Unknown data")
        
    n_data = raw_data.shape[0]
    indexes = np.random.choice(n_data, int(n_data), replace=False)
    train = np.random.choice(n_data, int(n_data * split), replace=False)
    test = np.setdiff1d(np.arange(n_data), train)
    test = np.random.choice(test, min(200, len(test)), replace=False) # limiting too large test set to avoid expense with OpenAI requests

    if N > len(train):
        raise ValueError(f"N must be less than the training set size. Trainin set size: {len(train)}")
    train_data = raw_data.iloc[train, :].reset_index(drop=True)[:N]
    test_data = raw_data.iloc[test, :].reset_index(drop=True)
    print(f"Dataset size:  \t{n_data}")
    print(f"Training size: \t{len(train_data)}")
    print(f"Test size:     \t{len(test_data)}")

    return raw_data, train_data, test_data, indexes, x_name, y_name


In [ ]:
def get_asktell(model: str, kwargs: dict = {}, pool: bolift.Pool = None, knn: int = 1):
    match model:
        case "mistral-7b-instruct:free":
            kwargs['model']="openrouter/mistralai/mistral-7b-instruct:free"
            return bolift.AskTellFewShotTopk(**kwargs)
        case "gemini-2.5-flash-preview":
            kwargs['model']="openrouter/google/gemini-2.5-flash-preview"
            return bolift.AskTellFewShotTopk(**kwargs)
        case "gpt-3.5-turbo-instruct":
            kwargs['model']="gpt-3.5-turbo-instruct"
            return bolift.AskTellFewShotTopk(**kwargs)
        case "gpt-3.5-turbo-0125":
            kwargs['model']="gpt-3.5-turbo-0125"
            return bolift.AskTellFewShotTopk(**kwargs)
        case "gpt-4":
            kwargs['model']="gpt-4"
            return bolift.AskTellFewShotTopk(**kwargs)
        case "gpt-4o":
            kwargs['model']="gpt-4o-2024-05-13"
            return bolift.AskTellFewShotTopk(**kwargs)
        case "gpt-4o-mini":
            kwargs['model']="gpt-4o-mini-2024-07-18"
            return bolift.AskTellFewShotTopk(**kwargs)
        case "haiku":
            kwargs['model']="claude-3-haiku-20240307"
            return bolift.AskTellFewShotTopk(**kwargs)
        case "sonnet":
            kwargs['model']="claude-3-5-sonnet-20240620"
            return bolift.AskTellFewShotTopk(**kwargs)
        case "opus":
            kwargs['model']="claude-3-opus-20240229"
            return bolift.AskTellFewShotTopk(**kwargs)
        case "gpt-4-0125-preview":
            kwargs['model']="gpt-4-0125-preview"
            return bolift.AskTellFewShotTopk(**kwargs)
        case "davinci":
            kwargs['model']="davinci-002"
            return bolift.AskTellFewShotTopk(**kwargs)
        case "gpr":
            s = kwargs['selector_k']
            kwargs['selector_k'] = 0
            kwargs['pool'] = pool if pool else None
            kwargs['n_components'] = 32
            model = bolift.AskTellGPR(**kwargs) 
            del kwargs['pool']
            del kwargs['n_components']
            kwargs['selector_k'] = s
            return model
        case "knn":
            s = kwargs['selector_k']
            del kwargs['selector_k']
            kwargs['knn'] = knn
            model = bolift.AskTellNearestNeighbor(**kwargs)
            del kwargs['knn']
            kwargs['selector_k'] = s
            return model
        case "krr":
            kwargs['alpha'] = 0.5
            model = bolift.AskTellRidgeKernelRegression(**kwargs)
            del kwargs['alpha']
            return model
        case "finetune":
            s = kwargs['selector_k']
            del kwargs['selector_k']
            kwargs['model']="gpt-3.5-turbo"
            model = bolift.AskTellFinetuning(**kwargs)
            kwargs['selector_k'] = s
            return model
        case _:
            raise ValueError("Unknown model")

In [ ]:
def run_experiment(exp, dataset, system_message="", *args, **kwargs):
    T_list = exp['T_list']
    k_list = exp['k_list']
    N_list = exp['N_list']
    models_list = exp['models_list']
    out_csv_file = exp['out_csv_file']
    model_class="topk"
    for T, k, N, model in itertools.product(T_list, k_list, N_list, models_list):
        if model == "gpr" and N <= 5: continue # GPR needs at least 5 data points
        print(f"Running {dataset} {model_class} regression with T={T}, k={k}, N={N}, model={model}", end=" ")
        raw_data, train_data, test_data, indexes, x_name, y_name = get_dataset(dataset, N, split=0.8)
        kwargs['temperature'] = T
        kwargs['selector_k'] = k
        pool=None
        if model == "gpr":
            pool = bolift.Pool(raw_data[x_name].tolist())
        asktell = get_asktell(model, kwargs=kwargs, pool=pool, knn=5)
        x, y, yhat =  run_ablation_experiment(asktell, train_data, test_data, system_message=system_message)
        save_csv(out_csv_file, x, y, yhat, dataset, model, T, k, N, model_class, asktell.tokens_used)
        print(" --> done")

def save_backup(out_csv_file):
    if os.path.exists(f"{out_csv_file}.csv"):
        i = 1
        while os.path.exists(f"{out_csv_file}{i}.csv"):
            i += 1
        os.rename(f"{out_csv_file}.csv", f"{out_csv_file}{i}.csv")

def merge_exps(out_csv_file):
    save_backup(out_csv_file)
    all_files = glob.glob("regression-results_exp*.csv")
    exps = []

    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0, sep=";")
        exps.append(df)

    frame = pd.concat(exps, axis=0, ignore_index=True)
    frame.to_csv(f"{out_csv_file}.csv", index=False, sep=";")

    for filename in all_files:
        os.remove(filename)


## Config experiment

These values and datasets should be loaded accordingly with the experiments that are being done.
The union of all values considered in our experiments is available below.
    
```python
T_list = [0.01, 0.1, 0.7, 1.0]
k_list = [1, 2, 3, 4, 5]
N_list = [1, 2, 5, 10, 50, 100, 250, 500, 700, 1000]
models_list = ["gpt-3.5-turbo-instruct", "gpt-3.5-turbo-0125", "gpt-4-0125-preview", "gpt-4o" "davinci-002", "KNN", "RNN", "GPR", "FineTunning"]
out_csv_file = "regression_results.csv"
```

In [ ]:
exps = {
    "exp_1" : {
        "T_list" : [0.05],
        "k_list" : [1, 2, 5, 10],
        "N_list" : [1000],
        "models_list" : ["gpt-3.5-turbo-0125", "gpt-4-0125-preview", "gpt-4o"],
        "out_csv_file" : "regression-results1.csv",
    }
}

### in-house

In [ ]:
dataset="in-house"
kwargs = dict(
    prefix="",
    prompt_template=PromptTemplate(
        input_variables=["x", "y", "y_name"],
        template="Q: What is the {y_name} of {x}?@@@\nA: {y}###",
    ),
    suffix="What is the {y_name} of {x}?@@@\nA:",
    x_formatter=lambda x: f"experimental procedure: {x}",
    y_name="CO STY",
    y_formatter=lambda y: f"{y:.2f}",
)

### OCM

In [ ]:
dataset="ocm"
# dataset="biasfree_ocm"
kwargs = dict(
    # prefix="You are a bot who knows chemistry and catalysts. " \
    #         "Below, you'll see examples of experimental procedures to synthesize catalysts and the measured C2 yield in a oxidative methane coupling reaction. " \
    #         "The following question should be answered with a number and finished with ###\n",
    prefix="",
    prompt_template=PromptTemplate(
        input_variables=["x", "y", "y_name"],   
        template="Q: What is the {y_name} of {x}?@@@\nA: {y}###",
    ),
    suffix="What is the {y_name} of {x}?@@@\nA:",
    x_formatter=lambda x: f"experimental procedure: {x}",
    y_name="C2 yield",
    y_formatter=lambda y: f"{y:.2f}",
)

inv_system_message_path = "./prompts/inv_prompt_1.txt"
system_message_path = "./prompts/prompt_1.txt"

exps = {
    # "exp_1" : { # Experiment 1 -> Varying k
    #     "T_list" : [0.05],
    #     "k_list" : [1, 2, 5, 10],
    #     "N_list" : [1000],
    #     "models_list" : ["gpt-3.5-turbo-0125"],
    #     "out_csv_file" : "regression-results_exp1.csv",
    # },
    # "exp_2": { # Experiment 2 -> Varying T
    #     "T_list" : [0.01, 0.1, 0.5, 0.7, 1.0, 1.5],
    #     "k_list" : [5],
    #     "N_list" : [1000],
    #     "models_list" : ["gpt-3.5-turbo-0125"],
    #     "out_csv_file" : "regression-results_exp2.csv",
    # },
    "exp_3": { # Experiment 3 -> Varying N
        "T_list" : [0.7],
        "k_list" : [5],
        "N_list" : [1, 5, 10, 25, 50, 100, 250, 500, 1000],
        # "models_list" : ["gpt-3.5-turbo-0125"],
        "models_list" : ["gemini-2.5-flash-preview"],
        "out_csv_file" : "regression-results_exp3.csv",
    },
    # "exp_4": { # Experiment 4 -> Varying the model
    #     "T_list" : [0.7],
    #     "k_list" : [5],
    #     "N_list" : [1000],
    #     "models_list" : ["gpt-4o", "gpt-3.5-turbo-0125", "gpt-4-0125-preview", "gpr"],
    #     "out_csv_file" : "regression-results_exp4.csv",
    # }
}

### Alloy

In [ ]:
dataset="alloy"
# dataset="biasfree_ocm"

kwargs = dict(
    # prefix="You are a bot who knows chemistry and catalysts. " \
    #         "Below, you'll see examples of experimental procedures to synthesize catalysts and the measured C2 yield in a oxidative methane coupling reaction. " \
    #         "The following question should be answered with a number and finished with ###\n",
    prefix="",
    prompt_template=PromptTemplate(
        input_variables=["x", "y", "y_name"],
        template="Q: What is the {y_name} of {x}?@@@\nA: {y}###",
    ),
    suffix="What is the {y_name} of {x}?@@@\nA:",
    x_formatter=lambda x: f"the corresponding experimental procedure: {x}",
    y_name="the log(charge transfer) [coulombs/cm²]", # inverse prompt : If {y_name} is {y}, then {x_name} is @@@\n{x}###"
    y_formatter=lambda y: f"{y:.2f}",
    selector_k=5,
    temperature=0.7,
    use_logprobs=False
)

inv_system_message_path = "./prompts/gpt4_alloy_dataset_tale.txt"
system_message_path = "./prompts/prompt_2_alloy_binary.txt"

exps = {
    # "exp_1" : { # Experiment 1 -> Varying k
    #     "T_list" : [0.05],
    #     "k_list" : [1, 2, 5, 10],
    #     "N_list" : [1000],
    #     "models_list" : ["gpt-3.5-turbo-0125"],
    #     "out_csv_file" : "regression-results_exp1.csv",
    # },
    # "exp_2": { # Experiment 2 -> Varying T
    #     "T_list" : [0.01, 0.1, 0.5, 0.7, 1.0, 1.5],
    #     "k_list" : [5],
    #     "N_list" : [1000],
    #     "models_list" : ["gpt-3.5-turbo-0125"],
    #     "out_csv_file" : "regression-results_exp2.csv",
    # },
    # "exp_3": { # Experiment 3 -> Varying N
    #     "T_list" : [0.7],
    #     "k_list" : [5],
    #     "N_list" : [1, 5, 10, 25, 50, 100, 250, 500, 1000],
    #     "models_list" : ["gpt-3.5-turbo-0125"],
    #     "out_csv_file" : "regression-results_exp3.csv",
    # },
    "exp_4": {
        "T_list" : [0.7],
        "k_list" : [5],
        "N_list" : [10,100],
        "models_list" : ["davinci-002"],
        "out_csv_file" : "regression-results_exp4.csv",
    }
}

### Solubility

In [ ]:
dataset="sol"
kwargs = dict(
    prefix="",
    prompt_template=PromptTemplate(
        input_variables=["x", "y", "y_name"],
        template="Q: What is the {y_name} of {x}?@@@\nA: {y}###",
    ),
    suffix="What is the {y_name} of {x}?@@@\nA:",
    x_formatter=lambda x: f"iupac name {x}",
    y_name="measured log solubility in mols per litre",
    y_formatter=lambda y: f"{y:.2f}",
)


# inv_system_message_path = "./prompts/inv_prompt_1.txt"
system_message_path = "./prompts/prompt_sol.txt"

exps = {
    # "exp_1" : { # Experiment 1 -> Varying k
    #     "T_list" : [0.05],
    #     "k_list" : [1, 2, 5, 10],
    #     "N_list" : [700],
    #     "models_list" : ["gpt-3.5-turbo-0125"],
    #     "out_csv_file" : "regression-results_exp1.csv",
    # },
    # "exp_2": { # Experiment 2 -> Varying T
    #     "T_list" : [0.01, 0.1, 0.5, 1.0],
    #     "k_list" : [5],
    #     "N_list" : [700],
    #     "models_list" : ["gpt-3.5-turbo-0125"],
    #     "out_csv_file" : "regression-results_exp2.csv",
    # },
    # "exp_3": { # Experiment 3 -> Varying N
    #     "T_list" : [0.7],
    #     "k_list" : [5],
    #     "N_list" : [1, 5, 10, 25, 50, 100, 250, 500, 700],
    #     "models_list" : ["gpt-3.5-turbo-0125"],
    #     "out_csv_file" : "regression-results_exp3.csv",
    # },
    "exp_4": { # Experiment 4 -> Varying the model
        "T_list" : [0.7],
        "k_list" : [5],
        "N_list" : [700],
        # "models_list" : ["gpt-3.5-turbo-instruct", "davinci-002", "gpt-4-0125-preview"],
        "models_list" : ["knn", "krr"],
        "out_csv_file" : "regression-results_exp4_sol.csv",
    }
}

## Run Regression

In [ ]:
if os.path.exists(system_message_path):
    with open(system_message_path, "r") as f:
        system_message = f.read()

for i in range(5):
    print(f"Running experiment {i+1}")
    for e in exps:
        exp = exps[e]
        run_experiment(exp, dataset, system_message=system_message, **kwargs)

    merge_exps("gemini-regression")


In [ ]:
df = pd.read_csv("out/regression/gemini-regression1.csv", sep=";")
df = df[df.model == "gemini-2.5-flash-preview"]

# plot y by yhat
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
sns.set_context("talk")

plt.figure(figsize=(5, 5))
sns.scatterplot(data=df, x="y", y="yhat", hue="model", style="model")
plt.plot([-10, 20], [-10, 20], color="black", linestyle="--")
plt.xlabel("True value")
plt.ylabel("Predicted value")
plt.legend().remove()
plt.show()


In [ ]:
# Quick test of the last regression experiment
from matplotlib import pyplot as plt
from sklearn.metrics import mean_absolute_error


lim=(min(y)-1,max(y)+1)
# lim=(-20,40)
plt.plot(lim, lim)
plt.xlim(lim)
plt.ylim(lim)
# plt.scatter(X_test[:,0], [yhi.mean() for yhi in yhat], color="red", alpha=0.2)
# plt.scatter(X_test[:,0], y, color="blue", alpha=0.2)
# plt.scatter(X_train[:,0],train_data['completion'].to_list(), color="green", alpha=0.2)
plt.scatter(y, [yhi.mean() for yhi in yhat], color="red", alpha=0.2)
plt.scatter(train_data['completion'].to_list(),train_data['completion'].to_list())
# plt.errorbar(y, 
#             [yhi.mean() for yhi in yhat], 
#             yerr=[yhi.std() for yhi in yhat],
#             fmt='.', color='gray', alpha=0.4)
plt.text(lim[0] + 0.1*(max(y)-min(y)), lim[1] - 1*0.1*(max(y)-min(y)), f"correlation = {np.corrcoef(y, [yhi.mean() for yhi in yhat])[0,1]:.3f}")
plt.text(lim[0] + 0.1*(max(y)-min(y)), lim[1] - 2*0.1*(max(y)-min(y)), f"MAE = {mean_absolute_error(y, [yhi.mean() for yhi in yhat]):.3f}")

# Plot results

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
import uncertainty_toolbox as uct
import matplotlib as mpl
import matplotlib.font_manager as font_manager
urllib.request.urlretrieve('https://github.com/google/fonts/raw/main/ofl/ibmplexmono/IBMPlexMono-Regular.ttf', 'IBMPlexMono-Regular.ttf')
fe = font_manager.FontEntry(
    fname='IBMPlexMono-Regular.ttf',
    name='plexmono')
font_manager.fontManager.ttflist.append(fe)
plt.rcParams.update({'axes.facecolor':'#f5f4e9',
            'grid.color' : '#AAAAAA',
            'axes.edgecolor':'#333333',
            'figure.facecolor':'#FFFFFF',
            'axes.grid': False,
            'axes.prop_cycle':   plt.cycler('color', plt.cm.Dark2.colors),
            'font.family': fe.name,
            'figure.figsize': (3.5,3.5 / 1.2),
            'ytick.left': True,
            'xtick.bottom': True
           })

## Loading results

In [ ]:
data = "sol"
max_N= 700

# Load and merge all 5 dataframes
dfs = []
for i in range(1, 6):
    file_path = f"out/regression/{data}-regression-{i}.csv"
    try:
        temp_df = pd.read_csv(file_path, sep=';')
        temp_df['source'] = f"run_{i}"  # Add file origin
        dfs.append(temp_df)
    except FileNotFoundError:
        print(f"File not found: {file_path}")

# Merge all dataframes
all_df = pd.concat(dfs, ignore_index=True)
df.columns

In [ ]:
all_df[(all_df['data'] == data)].groupby(['Temperature', 'data', 'k_selected', 'model_class', "N_train", "model"]).size().reset_index().sort_values(by=["model_class", "Temperature"])


## Plotting

### Utils

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import log_loss

def mse(y, pred):
  # return np.mean((y-pred)**2)
  return mean_squared_error(y, pred)

def mae(y, pred):
  return mean_absolute_error(y, pred)

def r2(y, pred):
  return r2_score(y, pred)

def corr(y, pred):
  return np.corrcoef(y, pred)[0,1]

def acc(y, pred, threshold):
  acc = sum((abs(pred - y)<threshold))/len(pred)
  return acc

def log_likelihood(y, pred, ystd, eps=0):
  y = np.array(y)
  pred = np.array(pred)
  ystd = np.array(ystd)
  yvar = ystd**2 + eps
  neg_ll = 0.5 * (np.log(yvar) + ((y - pred)**2 / yvar))
  return np.sum(neg_ll)/len(y)

def select_df(df, data, k, T, model, model_class, N):
  config = {'k': k,
            'T': T,
            'data': data,
            'model': model,
            'model_class': model_class,
            'N': N,
            }

  q = f""
  if T != 'any':
    q += f"Temperature=={T} and "
  if k != 'any':  
    q+= f"k_selected=={k} and "
  if model != 'any': 
    q+= f"model=='{model}' and "
  q+= f"model_class=='{model_class}' and "
  if N != 'any':
    q+= f"N_train=={N} and "
  q += f"data=='{data}'"
  sel = df.query(q)
  if sel.empty:
    raise ValueError(f"Dataframe is empty for the configuration {config}")
  return sel

In [ ]:
def plot_parities(df, data_property, data_range, nrows, ncols, data=None, k=None, T=None, model=None, model_class=None, N=None, axis_name=None, calibration=None, recal_ind=1, out_name=None, GPR=False):
  config = {'k': k,
            'T': T,
            'data': data,
            'model': model,
            'model_class': model_class,
            'N': N,
            }

  if sum([1 for i in config.values() if i is None]) > 1:
    raise ValueError("Only the property being varied in data_range can me passed as None.")

  if nrows*ncols < len(data_range):
    raise ValueError('''There's not enough space to plat all data in data_range.
    Decrease the size of data_range or increase ncols or nrows.''')

  fig, axs = plt.subplots(nrows=nrows, ncols=ncols, sharey=False, figsize=(12, 4), dpi=600)
  for i, p in enumerate(data_range):
    config[data_property] = p
    y=[]
    yhat=[]
    yprob=[]
    ax = axs if ncols*nrows == 1 else axs.flatten()[i]

    df_sel = select_df(df, **config)

    for prompt in df_sel['x'].unique():
        y.append(df_sel[df_sel['x']==prompt]['y'].unique()[0])
        # max_p = np.argmax(df_sel[df_sel['y']==d]['yprobs'].values)
        yhat.append(df_sel[df_sel['x']==prompt]['yhat'].values)
        yprob.append(df_sel[df_sel['x']==prompt]['yprobs'].values)
    yprobs = [yhi.std() for yhi in yhat]
    if GPR:
        ymeans = np.array([yhi.mean() for yhi in yhat])
        ystds = np.array([ypi.mean() for ypi in yprob])
    else:
        ymeans = np.array([
                  np.sum(yhi*ypi) if len(yhi)>1 else yhi.mean()
                  for yhi,ypi in zip(yhat, yprob)
                ])
        ystds = np.array([
                  np.sqrt(np.sum((yhi-ymi)**2*ypi)) if np.sum((yhi-ymi)**2*ypi)>1 else 0.1 #ypi.mean()
                  for yhi,ypi,ymi in zip(yhat, yprob, ymeans)
                ])

    if calibration:
        if calibration == "scaling_factor":
          std_scaling = uct.recalibration.optimize_recalibration_ratio(ymeans[:recal_ind], ystds[:recal_ind], np.array(y[:recal_ind]),
                                                                        criterion="miscal")
          ystds = ystds * std_scaling
        elif calibration == "isotonic":
          exp_props, obs_props= uct.metrics_calibration.get_proportion_lists_vectorized(ymeans[:recal_ind], ystds[:recal_ind], np.array(y[:recal_ind]))
          recal_model = uct.recalibration.iso_recal(exp_props, obs_props)
          recal_bounds = uct.metrics_calibration.get_prediction_interval(ymeans, ystds, 0.95, recal_model)
          ystds=np.array([ymeans - recal_bounds.lower,
                 recal_bounds.upper - ymeans])

    ax.plot(y,y)
    ax.errorbar(y, 
                ymeans, 
                yerr=ystds,
                fmt='.', color='gray', alpha=0.3)
    ax.scatter(
        y, ymeans, s=6, alpha=1, color=f"C{i}"
    )
    
    ax.set_title(f"{data_property}={p}")

    lim = (min(y), max(y))
    
    if model in ["krr", "knn"]:
       print(ystds)
       metrics = {
          "accuracy": uct.metrics.get_all_accuracy_metrics(ymeans, np.array(y), verbose=False)
                  }
    else:
        metrics = uct.metrics.get_all_metrics(ymeans, ystds, np.array(y), verbose=False)
    ax.text(lim[0] + 0.1*(max(y)-min(y)), lim[1] - 1*0.1*(max(y)-min(y)), f"$(\\uparrow$)correlation = {metrics['accuracy']['corr']:.3f}")
    if model_class not in ["KRR", "KNN"]:
      ax.text(lim[0] + 0.1*(max(y)-min(y)), lim[1] - 2*0.1*(max(y)-min(y)), f"$(\\downarrow$)neg-ll = {metrics['scoring_rule']['nll']:.3f}")
    ax.text(lim[0] + 0.1*(max(y)-min(y)), lim[1] - 3*0.1*(max(y)-min(y)), f"$(\\downarrow$)MAE = {metrics['accuracy']['mae']:.3f}")

    ax.set_ylim(lim[0],lim[1])
    ax.set_xlim(lim[0],lim[1])

    ax.set_xlabel(f"measured {axis_name}")
    if (i%ncols==0):
      ax.set_ylabel(f"predicted {axis_name}")

  # plt.tight_layout()
  plt.show()
  # if (out_name):
  #   plt.savefig(f"figs/{out_name}", dpi=300)

In [ ]:
def plot_ablation(df, data_property, data_range, nrows, ncols, data=None, k=None, T=None, model=None, model_class=None, N=None, out_name=None, GPR=False):
  config = {'k': k,
            'T': T,
            'data': data,
            'model': model,
            'model_class': model_class,
            'N': N,
            }

  MAE_list = []
  RMSE_list = []
  r_list = []
  nll_list = []
  prop_list = []
  for i, p in enumerate(data_range):
    config[data_property] = p
    y=[]
    yhat=[]
    yprobs=[]
    yprob=[]	

    df_sel = select_df(df, **config)

    for prompt in df_sel['x'].unique():
        y.append(df_sel[df_sel['x']==prompt]['y'].unique()[0])
        # max_p = np.argmax(df_sel[df_sel['y']==d]['yprobs'].values)
        yhat.append(df_sel[df_sel['x']==prompt]['yhat'].values)
        yprob.append(df_sel[df_sel['x']==prompt]['yprobs'].values)
    yprobs = [yhi.std() for yhi in yhat]
    if GPR:
        ymeans = np.array([yhi.mean() for yhi in yhat])
        ystds = np.array([ypi.mean() for ypi in yprob])
    else:
        ymeans = np.array([
                  np.sum(yhi*ypi) if len(yhi)>1 else yhi.mean()
                  for yhi,ypi in zip(yhat, yprob)
                ])
        ystds = np.array([
                  np.sqrt(np.sum((yhi-ymi)**2*ypi)) if yhi.std()>1 else ypi.mean()
                  for yhi,ypi,ymi in zip(yhat, yprob, ymeans)
                ])

    metrics = uct.metrics.get_all_metrics(ymeans, ystds, np.array(y), verbose=False)
    r_list.append(metrics['accuracy']['corr'])
    RMSE_list.append(metrics['accuracy']['rmse'])
    MAE_list.append(metrics['accuracy']['mae'])
    nll_list.append(metrics['scoring_rule']['nll'])
    prop_list.append(p)
    print(f"{model_class}(N:{config['N']}/k:{config['k']}/T:{config['T']}) => RMSE: | MAE: {MAE_list[-1]} | r: {r_list[-1]} | nll: {nll_list[-1]}")

  fig, axs = plt.subplots(nrows=nrows, ncols=ncols, sharey=False, figsize=(4*ncols, 4*nrows), dpi=300)
  
  axs[0].plot(prop_list, MAE_list)
  axs[0].set_xlabel(data_property)
  axs[0].set_ylabel("$\\rightarrow$MAE")
  
  axs[1].plot(prop_list, r_list)
  axs[1].set_xlabel(data_property)
  axs[1].set_ylabel("$\\leftarrow$correlation")

  axs[2].plot(prop_list, nll_list)
  axs[2].set_xlabel(data_property)
  axs[2].set_yscale('log')
  axs[2].set_ylabel("$\\rightarrow$negative log-likelihood")

  plt.tight_layout()
  # plt.show()
  if (out_name):
    plt.savefig(f"figs/{out_name}", dpi=300)

In [ ]:
def get_sub_ablation_data(df, data_property, data_range, data=None, k=None, T=None, model=None, model_class=None, N=None, GPR=False):
  config = {'k': k,
          'T': T,
          'data': data,
          'model': model,
          'model_class': model_class,
          'N': N,
          }
   
  MAE_list = []
  RMSE_list = []
  r_list = []
  nll_list = []
  prop_list = []
  for i, p in enumerate(data_range):
    config[data_property] = p
    y=[]
    yhat=[]
    yprobs=[]
    yprob=[]

    df_sel = select_df(df, **config)

    for prompt in df_sel['x'].unique():
        y.append(df_sel[df_sel['x']==prompt]['y'].unique()[0])
        # max_p = np.argmax(df_sel[df_sel['y']==d]['yprobs'].values)
        yhat.append(df_sel[df_sel['x']==prompt]['yhat'].values)
        yprob.append(df_sel[df_sel['x']==prompt]['yprobs'].values)
    yprobs = [yhi.std() for yhi in yhat]
    if GPR:
        ymeans = np.array([yhi.mean() for yhi in yhat])
        ystds = np.array([ypi.mean() for ypi in yprob])
    else:
        ymeans = np.array([
                  np.sum(yhi*ypi) if len(yhi)>1 else yhi.mean()
                  for yhi,ypi in zip(yhat, yprob)
                ])
        ystds = np.array([
                  np.sqrt(np.sum((yhi-ymi)**2*ypi)) if np.sum((yhi-ymi)**2*ypi)>0 else 10
                  for yhi,ypi,ymi in zip(yhat, yprob, ymeans)
                ])

    if model_class in ["KRR", "KNN"]:
       metrics = {
          "accuracy": uct.metrics.get_all_accuracy_metrics(ymeans, np.array(y), verbose=False)
                  }
    else:
      metrics = uct.metrics.get_all_metrics(ymeans, ystds, np.array(y), verbose=False)
      nll_list.append(metrics['scoring_rule']['nll'])
    r_list.append(metrics['accuracy']['corr'])
    RMSE_list.append(metrics['accuracy']['rmse'])
    MAE_list.append(metrics['accuracy']['mae'])
    prop_list.append(p)
    with open("Table.tex", "a") as t:
      t.write(f"{config['data']}&{model_class}&{model}&{config['T']}&{config['k']}&{config['N']}&{RMSE_list[-1]}&{MAE_list[-1]}&{r_list[-1]}&{nll_list[-1] if nll_list else '-'}&\\\\\n")
    # print(f"{model_class}(N:{config['N']}/k:{config['k']}/T:{config['T']}) => RMSE: | MAE: {MAE_list[-1]} | r: {r_list[-1]} | nll: {nll_list[-1]}")
   
  return prop_list, MAE_list, r_list, nll_list

def create_sub_ablation(axs, df, lims, data_property, data_range, color='C0', data=None, k=None, T=None, model=None, model_class=None, N=None, label=False, GPR=False):
  config = {'k': k,
            'T': T,
            'data': data,
            'model': model,
            'model_class': model_class,
            'N': N,
            }

  prop_list, MAE_list, r_list, nll_list = get_sub_ablation_data(df, data_property, data_range, **config, GPR=GPR)

  for ax in axs:
    ax.label_outer()

  if label:
    if model_class=="GPR-BOT":
      axs[0].plot(prop_list, MAE_list, label="GPR", color=color)
    else:
      axs[0].plot(prop_list, MAE_list, label=model_class, color=color)
  else:
    axs[0].plot(prop_list, MAE_list, color=color)
  axs[0].set_ylabel("MAE\n$\leftarrow$")
  axs[0].set_ylim(lims[0])
  axs[0].set_label(model_class)
  
  axs[1].plot(prop_list, r_list, color=color)
  axs[1].set_xlabel(data_property)
  axs[1].set_ylabel("r\n$\\rightarrow$")
  axs[1].set_ylim(lims[1])
  axs[1].set_label(model_class)

  if False: #model_class not in ["KRR", "KNN"]:
    axs[2].plot(prop_list, nll_list, color=color)
    axs[2].set_xlabel(data_property)
    axs[2].set_yscale('log')
    axs[2].set_ylabel("neg-ll\n$\leftarrow$")
    # axs[1].set_label(model_class)

  for ax in axs:
    ax.label_outer()

In [ ]:
def create_sub_shadow(axs, df, lims, data_property, data_range, color='C0', data=None, k=None, T=None, model=None, model_class=None, N=None, label=False, GPR=False):

    all_MAE = []
    all_r = []
    config = {'k': k,
            'T': T,
            'data': data,
            'model': model,
            'model_class': model_class,
            'N': N,
            }

    for i in range(1, 6):  # Loop through the 5 CSV files
        df = pd.read_csv(f"out/regression/{data}-regression-{i}.csv", sep=';')
        df_sel = select_df(df, **config)
        
        prop_list, MAE_list, r_list, nll_list = get_sub_ablation_data(df_sel, data_property, data_range, data=data, k=k, T=T, model=model, model_class=model_class, N=N)

        all_MAE.append(MAE_list)
        all_r.append(r_list)

    all_MAE = np.array(all_MAE)
    all_r = np.array(all_r)

    # Compute average MAE and r
    avg_MAE = np.mean(all_MAE, axis=0)
    avg_r = np.mean(all_r, axis=0)

    # Compute min and max for uncertainty region
    min_MAE = np.min(all_MAE, axis=0)
    max_MAE = np.max(all_MAE, axis=0)
    min_r = np.min(all_r, axis=0)
    max_r = np.max(all_r, axis=0)

    # Plot average curve with uncertainty region
    axs[0].fill_between(prop_list, min_MAE, max_MAE, color=color, alpha=0.3)
    axs[0].plot(prop_list, avg_MAE, color=color)
    axs[0].set_ylabel("MAE\n$\leftarrow$")
    axs[0].set_ylim(lims[0])
    axs[0].legend()

    axs[1].fill_between(prop_list, min_r, max_r, color=color, alpha=0.3)
    axs[1].plot(prop_list, avg_r, color=color)
    axs[1].set_ylabel("r\n$\\rightarrow$")
    axs[1].set_ylim(lims[1])
    axs[1].legend()

    axs[1].set_xlabel(data_property)

    for ax in axs:
        ax.label_outer()

In [ ]:
def get_sub_parity_data():
   pass
   
   
def create_sub_parity(ax, df_sel, axis_name, model_class, lim=[-1,1], color='gray', GPR=False, title=None, calibration=None, recal_ind=0, show_metrics=True):
    y=[]
    yhat=[]
    yprob=[]
    for prompt in df_sel['x'].unique():
        y.append(df_sel[df_sel['x']==prompt]['y'].unique()[0])
        # max_p = np.argmax(df_sel[df_sel['y']==d]['yprobs'].values)
        yhat.append(df_sel[df_sel['x']==prompt]['yhat'].values)
        yprob.append(df_sel[df_sel['x']==prompt]['yprobs'].values)
    yprobs = [yhi.std() for yhi in yhat]
    if GPR:
        ymeans = np.array([yhi.mean() for yhi in yhat])
        ystds = np.array([ypi.mean() for ypi in yprob])
    else:
        ymeans = np.array([
                  np.sum(yhi*ypi) if len(yhi)>1 else yhi.mean()
                  for yhi,ypi in zip(yhat, yprob)
                ])
        ystds = np.array([
                  np.sqrt(np.sum((yhi-ymi)**2*ypi)) if len(yhi)>1 else ypi.mean()
                  for yhi,ypi,ymi in zip(yhat, yprob, ymeans)
                ])
        # hack to fix uncertainties in finetuned model. 3.559 is the training set (N=1000) std
        ystds = np.array([ysi if ysi!=10 else 3.559 for ysi in ystds]) 
  
    if calibration:
        if calibration == "scaling_factor":
          std_scaling = uct.recalibration.optimize_recalibration_ratio(ymeans[:recal_ind], ystds[:recal_ind], np.array(y[:recal_ind]),
                                                                        criterion="miscal")
          ystds = ystds * std_scaling
        elif calibration == "isotonic":
          exp_props, obs_props= uct.metrics_calibration.get_proportion_lists_vectorized(ymeans[:recal_ind], ystds[:recal_ind], np.array(y[:recal_ind]))
          recal_model = uct.recalibration.iso_recal(exp_props, obs_props)
          recal_bounds = uct.metrics_calibration.get_prediction_interval(ymeans, ystds, 0.95, recal_model)
          ystds=np.array([ymeans - recal_bounds.lower,
                 recal_bounds.upper - ymeans])

    if model_class in ["KRR", "KNN"] or calibration=="isotonic":
       metrics = {
          "accuracy": uct.metrics.get_all_accuracy_metrics(ymeans, np.array(y), verbose=False)
                  }
       metrics_to_return = [metrics['accuracy']['corr'], metrics['accuracy']['mae'], None]
    else:
      metrics = uct.metrics.get_all_metrics(ymeans, ystds, np.array(y), verbose=False)
      metrics_to_return = [metrics['accuracy']['corr'], metrics['accuracy']['mae'], metrics['scoring_rule']['nll']]
      if show_metrics:
        ax.text(lim[0] + 0.1*(max(y)-min(y)), lim[1] - 2*0.1*(max(y)-min(y)), f"$(\\downarrow$)neg-ll = {metrics_to_return[2]:.3f}")
    if show_metrics:
      ax.text(lim[0] + 0.1*(max(y)-min(y)), lim[1] - 1*0.1*(max(y)-min(y)), f"$(\\uparrow$)correlation = {metrics_to_return[1]:.3f}")
      ax.text(lim[0] + 0.1*(max(y)-min(y)), lim[1] - 3*0.1*(max(y)-min(y)), f"$(\\downarrow$)MAE = {metrics_to_return[0]:.3f}")

    # with open("Table.tex", "a") as t:
    #   t.write(f"{config['data']}&{model_class}&{model}&{config['T']}&{config['k']}&{config['N']}&{RMSE_list[-1]}&{MAE_list[-1]}&{r_list[-1]}&{nll_list[-1] if nll_list else '-'}&\\\\\n")
    # print(f"{model_class}(N:{config['N']}/k:{config['k']}/T:{config['T']}) => RMSE: | MAE: {MAE_list[-1]} | r: {r_list[-1]} | nll: {nll_list[-1]}")

    ax.set_xlabel(f"measured {axis_name}")
    ax.set_ylabel(f"predicted {axis_name}")
    ax.set_ylim(lim[0],lim[1])
    ax.set_xlim(lim[0],lim[1])
    ax.set_xticks(np.arange(lim[0],lim[1]+0.1,4.0))

    if title:
      ax.set_title(title)

    ax.plot(y,y)
    ax.plot(lim,lim)
    if model_class not in ["KRR", "KNN"]:
      ax.errorbar(y, 
                  ymeans, 
                  yerr=ystds,
                  fmt='.', color='gray', alpha=0.2)
    ax.scatter(
        y, ymeans, s=6, alpha=1, color=color
    )
    return metrics_to_return

### Paper Figures

In [ ]:
fig = plt.figure(figsize=(14,16), constrained_layout=True)
subfigs = fig.subfigures(1,1, wspace=0.1, hspace=0.1)

lims = [(0,5),(0,1),(-1,1)]

sub00 = subfigs.subplots(2,1, sharex=True, sharey=False)
df1 = all_df[all_df['source'] == f'run_1']
d01 = select_df(df1, data=data, k=5, T=0.7, model='gemini-2.5-flash-preview', model_class='topk', N='any')

create_sub_shadow(sub00, d01, lims, 'N', sorted(d01['N_train'].unique()), color='C1',data=data, k=5, T=0.7, model='gemini-2.5-flash-preview', model_class='topk', N='any', label=True)
d01 = select_df(df1, data=data, k=5, T=0.7, model='gpt-4o', model_class='topk', N='any')
create_sub_shadow(sub00, d01, lims, 'N', sorted(d01['N_train'].unique()), color='C2',data=data, k=5, T=0.7, model='gpt-4o', model_class='topk', N='any', label=True)
d01 = select_df(df1, data=data, k=5, T=0.7, model='knn', model_class='topk', N='any')
create_sub_shadow(sub00, d01, lims, 'N', sorted(d01['N_train'].unique()), color='C3',data=data, k=5, T=0.7, model='knn', model_class='topk', N='any', label=True)
d01 = select_df(df1, data=data, k=5, T=0.7, model='krr', model_class='topk', N='any')
create_sub_shadow(sub00, d01, lims, 'N', sorted(d01['N_train'].unique()), color='C4',data=data, k=5, T=0.7, model='krr', model_class='topk', N='any', label=True)
d01 = select_df(df1, data=data, k=5, T=0.7, model='gpr', model_class='topk', N='any')
create_sub_shadow(sub00, d01, lims, 'N', sorted(d01['N_train'].unique()), color='C5',data=data, k=5, T=0.7, model='gpr', model_class='topk', N='any', label=True)

fig.legend(loc='upper center', bbox_to_anchor=(0.5 ,0),
          fancybox=True, shadow=True)
plt.tight_layout()

In [ ]:
def make_shadow_models_fig():

  fig = plt.figure(figsize=(4,6), constrained_layout=True)
  subfigs = fig.subfigures(1,1, wspace=0.1, hspace=0.1) 

  lims = [(0,5),(0,1),(-1,1)]

  sub00 = subfigs.subplots(2,1, sharex=True, sharey=False)
  df0 = all_df[all_df['source'] == f'run_1']
  d00 = select_df(df0, data=data, k=5, T=0.7, model='gemini-2.5-flash-preview', model_class='topk', N='any')
  create_sub_shadow(sub00, d00, lims, 'N', sorted(d00['N_train'].unique()), data=data, color='C1', k=5, T=0.7, model='gemini-2.5-flash-preview', model_class='topk', N='any', label=False)
  d01 = select_df(df0, data=data, k=5, T=0.7, model='gpt-4o', model_class='topk', N='any')
  create_sub_shadow(sub00, d01, lims, 'N', sorted(d01['N_train'].unique()), data=data, color='C2', k=5, T=0.7, model='gpt-4o', model_class='topk', N='any', label=False)
  d02 = select_df(df0, data=data, k=5, T=0.7, model='knn', model_class='topk', N='any')
  create_sub_shadow(sub00, d02, lims, 'N', sorted(d02['N_train'].unique()), data=data, color='C3', k=5, T=0.7, model='knn', model_class='topk', N='any', label=False)
  d03 = select_df(df0, data=data, k=5, T=0.7, model='krr', model_class='topk', N='any')
  d03 = d03[d03['N_train']!=1]
  create_sub_shadow(sub00, d03, lims, 'N', sorted(d03['N_train'].unique()), data=data, color='C4', k=5, T=0.7, model='krr', model_class='topk', N='any', label=False)
  d04 = select_df(df0, data=data, k=5, T=0.7, model='gpr', model_class='topk', N='any')
  create_sub_shadow(sub00, d04, lims, 'N', sorted(d04['N_train'].unique()), data=data, color='C5', k=5, T=0.7, model='gpr', model_class='topk', N='any', label=False)

  fig.legend(loc='upper center', bbox_to_anchor=(0.5 ,0),
            fancybox=True, shadow=True, ncol=5)

  # plt.tight_layout()
  plt.savefig(f"./out/figs/{data}_metrics_models.png", dpi=300, bbox_inches='tight')
  plt.show()

make_shadow_models_fig()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def show_metrics(ax, metrics, lim):
  corr = [m[0] for m in metrics]
  mae = [m[1] for m in metrics]
  nll = [m[2] for m in metrics]
  ax.text(lim[0] + 0.01*(lim[1]-lim[0]), lim[1] - 1*0.1*(lim[1]-lim[0]), f"$(\\uparrow$)correlation = {np.mean(corr):.3f}±{np.std(corr):.3f}")
  ax.text(lim[0] + 0.01*(lim[1]-lim[0]), lim[1] - 2*0.1*(lim[1]-lim[0]), f"$(\\downarrow$)MAE = {np.mean(mae):.3f}±{np.std(mae):.3f}")
  if all([n is not None for n in nll]):
    ax.text(lim[0] + 0.01*(lim[1]-lim[0]), lim[1] - 3*0.1*(lim[1]-lim[0]), f"$(\\downarrow$)neg-ll = {np.mean(nll):.3f}±{np.std(nll):.3f}")

fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(12,8), constrained_layout=True)
for ax in axs.flat:
    ax.set_aspect(1)

gpt35_metrics = []
gemini_metrics = []
gpt4o_metrics = []
krr_metrics = []
finetune_metrics = []
gpr_metrics = []

for i in range(1,6):
    df = all_df[all_df['source'] == f'run_{i}']
    d00 = select_df(df, data=data, k=5, T=0.7, model='gpt-3.5-turbo-0125', model_class='topk', N=max_N)
    lim = (min(d00['y']), max(d00['y']))
    lim = (-2, 25)
    lim = (-15, 5) # Sol limits with some space
    text_anchor = sum(lim)/len(lim)
    gpt35_metrics.append(create_sub_parity(axs[0,0], d00, 'LogS solubility', lim=lim, model_class="topk", color=f'C0', title="gpt-3.5", show_metrics=False))
    d01 = select_df(df, data=data, k=5, T=0.7, model='gpt-4o-mini', model_class='topk', N=max_N)
    gemini_metrics.append(create_sub_parity(axs[0,1], d01, 'LogS solubility', lim=lim, model_class="topk", color=f'C1', title="gpt-4o-mini", show_metrics=False))
    d02 = select_df(df, data=data, k=5, T=0.7, model='gpt-4o', model_class='topk', N=max_N)
    gpt4o_metrics.append(create_sub_parity(axs[0,2], d02, 'LogS solubility', lim=lim, model_class="topk", color=f'C2', title="gpt-4o", show_metrics=False))

    if i not in [5]:
        d10 = select_df(df, data=data, k=5, T=0.7, model='krr', model_class='topk', N=max_N)
        krr_metrics.append(create_sub_parity(axs[1,0], d10, 'LogS solubility', lim=lim, model_class="KRR", color=f'C4', title="KRR", show_metrics=False))
    if i not in [1,2,5]:
        d11 = select_df(df, data=data, k=5, T=0.7, model='finetune', model_class='topk', N=max_N)
        finetune_metrics.append(create_sub_parity(axs[1,1], d11, 'LogS solubility', lim=lim, model_class="KRR", color=f'C5', title="finetune", show_metrics=False))
    d12 = select_df(df, data=data, k=5, T=0.7, model='gpr', model_class='topk', N=max_N)
    gpr_metrics.append(create_sub_parity(axs[1,2], d12, 'LogS solubility', lim=lim, model_class="topk", color=f'C6', title="GPR", show_metrics=False))

show_metrics(axs[0,0], gpt35_metrics, lim)
show_metrics(axs[0,1], gemini_metrics, lim)
show_metrics(axs[0,2], gpt4o_metrics, lim)
show_metrics(axs[1,0], krr_metrics, lim)
show_metrics(axs[1,1], finetune_metrics, lim)
show_metrics(axs[1,2], gpr_metrics, lim)

plt.savefig(f"./out/figs/{data}_par_models", dpi=300, bbox_inches='tight')
plt.show()

### C2

In [ ]:
c2_data = df[(df['data'] == 'ocm')]
c2_data.groupby(['Temperature', 'data', 'k_selected', 'model_class', "N_train", "model"]).size().reset_index().sort_values(by=["model_class", "Temperature"])

#### topk

In [ ]:
plot_parities(df, 
              'model', 
              ["gemini-2.5-flash-preview", "gpt-4-0125-preview", "gpt-4o"], #sorted(c2_data[(c2_data['model_class']=="topk") & (c2_data['model']=="text-curie-001")]['N_train'].unique()), 
              nrows=1, ncols=3,
              data='ocm', 
              k=5, 
              T=0.7, 
              model=None, 
              model_class='topk', 
              N=1000,
              calibration=None,
              recal_ind=300,
              axis_name="C2 yield",
              out_name="par_C2_topk_N_curie.png")

In [ ]:
plot_parities(df, 
              'N', 
              [1,50,250,1000], #sorted(c2_data[(c2_data['model_class']=="topk") & (c2_data['model']=="text-curie-001")]['N_train'].unique()), 
              nrows=1, ncols=4,
              data='ocm', 
              k=5, 
              T=0.7, 
              model='gpt-3.5-turbo-0125', 
              model_class='topk', 
              N=None,
              calibration=None,
              recal_ind=300,
              axis_name="C2 yield",
              out_name="par_C2_topk_N_curie.png")

plot_parities(df, 
              'k', 
              [1,2,5,10], #sorted(c2_data[(c2_data['model_class']=="topk") & (c2_data['model']=="text-curie-001")]['k_selected'].unique()), 
              nrows=1, ncols=4,
              data='ocm', 
              k=None, 
              T=0.05, 
              model='gpt-3.5-turbo-0125', 
              model_class='topk', 
              N=10,
              calibration=None,
              recal_ind=300,
              axis_name="C2 yield",
              out_name="par_C2_topk_k_curie.png")

plot_parities(df, 
              'T', 
              [0.01,0.1,0.5,1.0], #sorted(c2_data[(c2_data['model_class']=="topk") & (c2_data['model']=="text-curie-001")]['k_selected'].unique()), 
              nrows=1, ncols=4,
              data='ocm', 
              k=5, 
              T=None, 
              model='gpt-3.5-turbo-0125', 
              model_class='topk', 
              N=1000,
              calibration=None,
              recal_ind=300,
              axis_name="C2 yield",
              out_name="par_C2_topk_T_curie.png")

#### topk-davinci

In [ ]:
plot_parities(c2_data, 
              'T', 
              sorted(c2_data[(c2_data['model_class']=="topk") & (c2_data['model']=="text-davinci-003")]['Temperature'].unique()), 
              nrows=1, ncols=4,
              data='C2',
              k=5,
              T=None,
              model='text-davinci-003',
              model_class='topk',
              N=1000,
              calibration=None,
              recal_ind=300,
              axis_name="C2 yield",
              out_name="par_C2_multi_T_curie.png")

In [ ]:
# plot_ablation(c2_data, 
#               'N', 
#               sorted(c2_data[(c2_data['model_class']=="topk") & (c2_data['model']=="text-davinci-003")]['N_train'].unique()), 
#               nrows=1, ncols=3,
#               data='C2',
#               k=5,
#               T=0.05,
#               model='text-davinci-003',
#               model_class='topk',
#               N=None,
#               out_name="ablation_C2_topk_N_davinci.png")

# plot_ablation(c2_data, 
#               'T', 
#               sorted(c2_data[(c2_data['model_class']=="topk") & (c2_data['model']=="text-davinci-003")]['Temperature'].unique()), 
#               nrows=1, ncols=3,
#               data='C2',
#               k=5,
#               T=None,
#               model='text-davinci-003',
#               model_class='topk',
#               N=1000,
#               out_name="ablation_C2_multi_T_davinci.png")

#### GPR

In [ ]:
plot_parities(c2_data, 
              'N', 
              [1,10,250,500], #sorted(c2_data[(c2_data['model_class']=="GPR-BOT") & (c2_data['model']=="text-ada-001")]['N_train'].unique()), 
              nrows=1, ncols=4,
              data='C2', 
              k=32, 
              T=0.05, 
              model='text-ada-001', 
              model_class='GPR-BOT', 
              N=None,
              calibration=None,
              recal_ind=300,
              axis_name="C2 yield",
              out_name="par_C2_GPR_N.png",
              GPR=True)

In [ ]:
# plot_ablation(c2_data, 
#               'N', 
#               sorted(c2_data[(c2_data['model_class']=="GPR-BOT") & (c2_data['model']=="text-ada-001")]['N_train'].unique()), 
#               nrows=1, ncols=3,
#               data='C2',
#               k=32,
#               T=0.05,
#               model='text-ada-001',
#               model_class='GPR-BOT',
#               N=None,
#               out_name="ablation_C2_GPR_N_ada.png",
#               GPR=True)

# plot_ablation(c2_data, 
#               'k',
#               sorted(c2_data[(c2_data['model_class']=="GPR-BOT") & (c2_data['model']=="text-ada-001")]['k_selected'].unique()), 
#               nrows=1, ncols=3,
#               data='C2',
#               k=None,
#               T=0.05,
#               model='text-ada-001',
#               model_class='GPR-BOT',
#               N=500,
#               out_name="ablation_C2_GPR_N_ada.png",
#               GPR=True)

#### KNN

In [ ]:
plot_parities(c2_data, 
              'N', 
              [5,10,25,50,100,250,500,1000], #sorted(c2_data[(c2_data['model_class']=="GPR-BOT") & (c2_data['model']=="text-ada-001")]['N_train'].unique()), 
              nrows=2, ncols=4,
              data='C2', 
              k=1, 
              T=0.05, 
              model='text-ada-001', 
              model_class='KNN', 
              N=None,
              calibration=None,
              recal_ind=300,
              axis_name="C2 yield",
              out_name="par_C2_KNN_N.png",
              GPR=True)

#### finetune

In [ ]:
plot_parities(c2_data, 
              'N', 
              [50,100,250,1000], #sorted(c2_data[(c2_data['model_class']=="finetune")]['N_train'].unique()), 
              nrows=1, ncols=4,
              data='C2', 
              k=0, 
              T=0.05, 
              model='any', 
              model_class='finetune', 
              N=None,
              calibration=None,
              recal_ind=300,
              axis_name="C2 yield",
              out_name="par_C2_FT_N.png")

In [ ]:
# plot_ablation(df, 
#               'N', 
#               sorted(c2_data[(c2_data['model_class']=="finetune")]['N_train'].unique()), 
#               nrows=1, ncols=3,
#               data='C2',
#               k=0,
#               T=0.05,
#               model='any',
#               model_class='finetune',
#               N=None,
#               out_name="ablation_C2_FT_N_ada.png")

#### curie X davinci

In [ ]:
df_models = df[
    (df['model_class']=="topk") &
    ((df['model']=="text-curie-001") | (df['model']=="text-davinci-003") | (df['model']=="gpt-4")) &
    (df['k_selected']==5) &
    (df['Temperature']==0.7)
]
df_models
plot_parities(df_models, 
              'model', 
              [
                "text-curie-001",
                "text-davinci-003",
                "gpt-4"
              ], #sorted(c2_data[c2_data['model_class']=="multi"]['model'].unique()), 
              nrows=1, ncols=3,
              data='C2', 
              k=5,
              T=0.7,
              model=None,
              model_class='topk', 
              N=1000,
              calibration=None,
              recal_ind=300,
              axis_name="C2 yield",
              out_name="par_C2_topk_N_curieXdavinci.png")

### iupac sol

In [ ]:
iupac_sol_data = df[(df['data'] == 'sol')]
iupac_sol_data.groupby(['Temperature', 'data','k_selected', 'model_class', "N_train", "model"]).size().reset_index().sort_values(by=["model_class", "Temperature"])

#### topk

In [ ]:
plot_parities(iupac_sol_data, 
              'N', 
              [1,10,250,700], #sorted(iupac_sol_data[iupac_sol_data['model_class']=="topk"]['N_train'].unique()), 
              nrows=1, ncols=4,
              data='sol', 
              k=5, 
              T=0.7, 
              model='gpt-3.5-turbo-0125', 
              model_class='topk',
              N=None,
              calibration=None,
              recal_ind=300,
              axis_name="LogS solubility",
              out_name="par_sol_topk_N_curie.png")

plot_parities(iupac_sol_data, 
              'k', 
              [1,5,10], #sorted(iupac_sol_data[iupac_sol_data['model_class']=="multi"]['N_train'].unique()), 
              nrows=1, ncols=3,
              data='sol', 
              k=None,
              T=0.05, 
              model='gpt-3.5-turbo-0125', 
              model_class='topk', 
              N=700,
              calibration=None,
              recal_ind=300,
              axis_name="LogS solubility",
              out_name="par_sol_topk_k_curie.png")

plot_parities(iupac_sol_data, 
              'T', 
              [0.05, 0.5, 0.7, 1.0], #sorted(iupac_sol_data[iupac_sol_data['model_class']=="multi"]['N_train'].unique()), 
              nrows=1, ncols=4,
              data='sol', 
              k=5,
              T=None, 
              model='gpt-3.5-turbo-0125', 
              model_class='topk', 
              N=700,
              calibration=None,
              recal_ind=300,
              axis_name="LogS solubility",
              out_name="par_sol_topk_T_curie.png")

In [ ]:
plot_ablation(iupac_sol_data, 
              'N', 
              sorted(iupac_sol_data[iupac_sol_data['model_class']=="topk"]['N_train'].unique()), 
              nrows=1, ncols=3,
              data='iupac-sol',
              k=5,
              T=0.05,
              model='text-curie-001',
              model_class='topk',
              N=None,
              out_name="ablation_sol_topk_N_curie.png")

plot_ablation(iupac_sol_data, 
              'k', 
              [1,5,10], #sorted(iupac_sol_data[iupac_sol_data['model_class']=="topk"]['k_selected'].unique()), 
              nrows=1, ncols=3,
              data='iupac-sol',
              k=None,
              T=0.05,
              model='text-curie-001',
              model_class='topk',
              N=700,
              out_name="ablation_sol_topk_k_curie.png")

plot_ablation(iupac_sol_data, 
              'T', 
              sorted(iupac_sol_data[iupac_sol_data['model_class']=="topk"]['Temperature'].unique()), 
              nrows=1, ncols=3,
              data='iupac-sol',
              k=5,
              T=None,
              model='text-curie-001',
              model_class='topk',
              N=700,
              out_name="ablation_sol_topk_T_curie.png")

#### GPR

In [ ]:
plot_parities(iupac_sol_data, 
              'N', 
              [1,10,250,700], #sorted(iupac_sol_data[(iupac_sol_data['model_class']=="GPR-BOT") & (iupac_sol_data['model']=="text-ada-001")]['N_train'].unique()), 
              nrows=1, ncols=4,
              data='iupac-sol', 
              k=32, 
              T=0.05, 
              model='text-ada-001', 
              model_class='GPR-BOT', 
              N=None,
              calibration=None,
              recal_ind=300,
              axis_name="LoS solubility",
              out_name="par_sol_GPR_N.png",
              GPR=True)

In [ ]:
plot_ablation(df, 
              'N', 
              sorted(iupac_sol_data[(iupac_sol_data['model_class']=="GPR-BOT") & (iupac_sol_data['model']=="text-ada-001")]['N_train'].unique()), 
              nrows=1, ncols=3,
              data='iupac-sol',
              k=32,
              T=0.05,
              model='text-ada-001',
              model_class='GPR-BOT',
              N=None,
              out_name="ablation_sol_GPR_N_ada.png")

#### KNN

In [ ]:
plot_parities(iupac_sol_data, 
              'N', 
              [5,10,25,50,100,250,500,700], #sorted(c2_data[(c2_data['model_class']=="GPR-BOT") & (c2_data['model']=="text-ada-001")]['N_train'].unique()), 
              nrows=2, ncols=4,
              data='iupac-sol', 
              k=1, 
              T=0.05, 
              model='text-ada-001', 
              model_class='KNN', 
              N=None,
              calibration=None,
              recal_ind=300,
              axis_name="C2 yield",
              out_name="par_sol_KNN_N.png",
              GPR=True)

#### finetune

In [ ]:
plot_parities(iupac_sol_data, 
              'N', 
              [50, 250, 700],#sorted(iupac_sol_data[iupac_sol_data['model_class']=="finetune"]['N_train'].unique()), 
              nrows=1, ncols=3,
              data='iupac-sol', 
              k=0, 
              T=0.05, 
              model='any', 
              N=None,
              axis_name="LogS solubility",
              out_name="par_sol_FT_N.png")

In [ ]:
plot_ablation(iupac_sol_data, 
              'N', 
              sorted(iupac_sol_data[iupac_sol_data['model_class']=="finetune"]['N_train'].unique()), 
              nrows=1, ncols=3,
              data='iupac-sol',
              k=0,
              T=0.05,
              model='any',
              model_class='finetune',
              N=None,
              out_name="ablation_sol_FT_N_ada.png")

### alloy

In [ ]:
alloy_data = df[(df['data'] == 'alloy')]
alloy_data.groupby(['Temperature', 'k_selected', 'model_class', "N_train", "model"]).size().reset_index().sort_values(by=["model_class", "Temperature"])

### Calibrated MMR vs Sim

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(14,12), constrained_layout=True)
for ax in axs.flat:
    ax.set_aspect(1)

d00 = select_df(df, data="C2", k=5, T=0.7, model='text-curie-001', model_class='topk_NN', N=1000)
lim_c2 = (min(d00['y']), max(d00['y']))
lim_c2 = (-2, 25)
text_anchor = sum(lim_c2)/len(lim_c2)
create_sub_parity(axs[0], d00, 'C2 yield', lim=lim_c2, model_class="topk_NN", color=f'C4', title="topk_cos_sim|N=1000|T=.7|k=5|curie",calibration='scaling_factor',recal_ind=300)

d00 = select_df(df, data="C2", k=5, T=0.7, model='text-curie-001', model_class='multi_NN', N=1000)
lim_c2 = (min(d00['y']), max(d00['y']))
lim_c2 = (-2, 25)
text_anchor = sum(lim_c2)/len(lim_c2)
create_sub_parity(axs[1], d00, 'C2 yield', lim=lim_c2, model_class="multi_NN", color=f'C4', title="multi_cos_sim|N=1000|T=.7|k=5|curie",calibration='scaling_factor',recal_ind=300)


plt.show()

### Calibration Curves

In [ ]:


def create_sub_parity(ax, df_sel, axis_name="topk",model_class="topk", lim=[-1,1], color='gray', GPR=False, Type_cali =False,rec_split=100,model='curie-001'):
   
    def process_data(df, unique_vals):
        y, yhat, yprob = [], [], []
        for prompt in unique_vals:
            y.append(df[df['x'] == prompt]['y'].unique()[0])
            yhat.append(df[df['x'] == prompt]['yhat'].values)
            yprob.append(df[df['x'] == prompt]['yprobs'].values)
        return y, yhat, yprob

    def calculate_means_and_stds(yhat, yprob):
        means = np.array([np.sum(yhi * ypi) if len(yhi) > 1 else ypi.mean() for yhi, ypi in zip(yhat, yprob)])
        means = np.where(means == 0, 0.0001, means)
        stds = np.array([np.sqrt(np.sum((yhi - ymi)**2 * ypi)) if len(yhi) > 1 else ypi.mean() for yhi, ypi, ymi in zip(yhat, yprob, means)])
        return means, stds

    unique_vals = df_sel['x'].unique()
    y, yhat, yprob = process_data(df_sel, unique_vals[:rec_split])
    y_rec, yhat_rec, yprob_rec = process_data(df_sel, unique_vals[rec_split:])

    ymeans, ystds = calculate_means_and_stds(yhat, yprob)
    ymeans_rec, ystds_rec = calculate_means_and_stds(yhat_rec, yprob_rec)
    
    if Type_cali == "cali":

        if GPR:
            yprobs = np.array([ypi.mean() for ypi in yprob])

            yhats=np.array(np.concatenate(yhat))

            ma = uct.miscalibration_area(yhats,yprobs, np.array(y), recal_model=None)
            
            x, y1 = uct.get_proportion_lists_vectorized(yhats,yprobs, np.array(y))
            
            ax.plot(x,x, linestyle= '--')
            ax.plot(x, y1)
            ax.fill_between(x, x, y1, color='teal', alpha=0.3)
            
            ax.set_title(' {} | {} | {}'.format(model,axis_name, model_class))
           
            ax.set_ylim(lim[0],lim[1])
            ax.set_xlim(lim[0],lim[1])
            ax.set_ylabel("Observed Proportion in Interval")
            
            # ax.(lim[0] + 0.1*(max(yhat)-min(yhat)), lim[1] - 1*0.1*(max(yhat)-min(y)), f"(\u2193)Miscalibration area = {ma:.3f}")

            ax.annotate(f"(\u2193)Miscalibration area = {ma:.3f}",
                xy=(0.02, 0.98), xycoords='axes fraction',
                horizontalalignment='left', verticalalignment='top')

        else:
            ma = uct.miscalibration_area(ymeans,ystds, np.array(y), recal_model=None)
            
            x, y1 = uct.get_proportion_lists_vectorized(ymeans,ystds, np.array(y))
            
            ax.plot(x,x, linestyle= '--')
            ax.plot(x, y1)
            ax.fill_between(x, x, y1, color='teal', alpha=0.3)
            
            ax.set_title(' {} | {} | {}'.format(model,axis_name, model_class))
            # ax.set_xlabel(f"measured {axis_name}")
            # ax.set_ylabel("Obsererved Proportion in Interval")
            ax.set_ylabel("Observed Proportion in Interval")
            ax.set_ylim(lim[0],lim[1])
            ax.set_xlim(lim[0],lim[1])
            
            # ax.(lim[0] + 0.1*(max(yhat)-min(yhat)), lim[1] - 1*0.1*(max(yhat)-min(y)), f"(\u2193)Miscalibration area = {ma:.3f}")

            ax.annotate(f"(\u2193)Miscalibration area = {ma:.3f}",
                xy=(0.02, 0.98), xycoords='axes fraction',
                horizontalalignment='left', verticalalignment='top')
            

    elif Type_cali == "recali":

        if GPR:

            yprobs = np.array([ypi.mean() for ypi in yprob])
            yyprobs = np.array([ypi.mean() for ypi in yprob_rec])

            yhats=np.array(np.concatenate(yhat))
            yyhats=np.array(np.concatenate(yhat_rec))


            exp_props,obs_props= uct.metrics_calibration.get_proportion_lists_vectorized(yyhats[:100], yyprobs[:100], np.array(y_rec[:100]))

            
            recal_model = uct.recalibration.iso_recal(exp_props, obs_props)

            ma = uct.miscalibration_area(yhats, yprobs, np.array(y), recal_model=recal_model)
            
            x, y1 = uct.metrics_calibration.get_proportion_lists_vectorized(yhats, yprobs, np.array(y), recal_model=recal_model)
            
            ax.plot(x,x, linestyle= '--')
            ax.plot(x, y1)
            ax.fill_between(x, x, y1, color='teal', alpha=0.3)
            
            ax.set_title('Isotonic')
            ax.set_ylim(lim[0],lim[1])
            ax.set_xlim(lim[0],lim[1])
            
            # ax.(lim[0] + 0.1*(max(yhat)-min(yhat)), lim[1] - 1*0.1*(max(yhat)-min(y)), f"(\u2193)Miscalibration area = {ma:.3f}")

            ax.annotate(f"(\u2193)Miscalibration area = {ma:.3f}",
                xy=(0.02, 0.98), xycoords='axes fraction',
                horizontalalignment='left', verticalalignment='top')

        else:
        
            exp_props,obs_props= uct.metrics_calibration.get_proportion_lists_vectorized(ymeans_rec[:100], ystds_rec[:100], np.array(y_rec[:100]))


            recal_model = uct.recalibration.iso_recal(exp_props, obs_props)

            ma = uct.miscalibration_area(ymeans, ystds, np.array(y), recal_model=recal_model)
            
            x, y1 = uct.metrics_calibration.get_proportion_lists_vectorized(ymeans,ystds, np.array(y), recal_model=recal_model)
            
            ax.plot(x,x, linestyle= '--')
            ax.plot(x, y1)
            ax.fill_between(x, x, y1, color='teal', alpha=0.3)
            
            ax.set_title('Isotonic')
            
            ax.set_ylabel("Observed Proportion in Interval")
            ax.set_ylim(lim[0],lim[1])
            ax.set_xlim(lim[0],lim[1])
            
            # ax.(lim[0] + 0.1*(max(yhat)-min(yhat)), lim[1] - 1*0.1*(max(yhat)-min(y)), f"(\u2193)Miscalibration area = {ma:.3f}")

            ax.annotate(f"(\u2193)Miscalibration area = {ma:.3f}",
                xy=(0.02, 0.98), xycoords='axes fraction',
                horizontalalignment='left', verticalalignment='top')

    elif Type_cali == "recali_scale":


        if GPR:
            
            yhats=np.concatenate(yhat)
            yyhats=np.concatenate(yhat_rec)

            yprobs = np.array([ypi.mean() for ypi in yprob])
            yyprobs = np.array([ypi.mean() for ypi in yprob_rec])

            std_scaling = uct.recalibration.optimize_recalibration_ratio(yyhats[:100], yyprobs[:100], np.array(y_rec[:100]), criterion="miscal")


            ystds = yprobs * std_scaling
            print(std_scaling,model)

            ma = uct.miscalibration_area(yhats, ystds, np.array(y), recal_model=None)
    
            
            x, y1 = uct.metrics_calibration.get_proportion_lists_vectorized(yhats, ystds, np.array(y), recal_model=None)
            
            ax.plot(x,x, linestyle= '--')
            ax.plot(x, y1)
            ax.fill_between(x, x, y1, color='teal', alpha=0.3)
            
            ax.set_title('Scaling')
            ax.set_xlabel("Predicted Proportion in Interval")
            ax.set_ylabel("Observed Proportion in Interval")
            ax.set_ylim(lim[0],lim[1])
            ax.set_xlim(lim[0],lim[1])
            

            ax.annotate(f"(\u2193)Miscalibration area = {ma:.3f}",
                xy=(0.02, 0.98), xycoords='axes fraction',
                horizontalalignment='left', verticalalignment='top')

        else:

            std_scaling = uct.recalibration.optimize_recalibration_ratio(ymeans_rec[:100], ystds_rec[:100], np.array(y_rec[:100]),
                                                                        criterion="miscal")
            print(std_scaling)
            ystds = ystds * std_scaling


            ma = uct.miscalibration_area(ymeans, ystds, np.array(y), recal_model=None)
            
            x, y1 = uct.metrics_calibration.get_proportion_lists_vectorized(ymeans, np.array(ystds), np.array(y), recal_model=None)
            
            ax.plot(x,x, linestyle= '--')
            ax.plot(x, y1)
            ax.fill_between(x, x, y1, color='teal', alpha=0.3)
            
            ax.set_title('Scaling')
            ax.set_xlabel("Predicted Proportion in Interval")
            ax.set_ylabel("Observed Proportion in Interval")
            ax.set_ylim(lim[0],lim[1])
            ax.set_xlim(lim[0],lim[1])
            
            # ax.(lim[0] + 0.1*(max(yhat)-min(yhat)), lim[1] - 1*0.1*(max(yhat)-min(y)), f"(\u2193)Miscalibration area = {ma:.3f}")

            ax.annotate(f"(\u2193)Miscalibration area = {ma:.3f}",
                xy=(0.02, 0.98), xycoords='axes fraction',
                horizontalalignment='left', verticalalignment='top')

    else:

        ax.set_xlabel(f"measured {axis_name}")
        ax.set_ylabel(f"predicted {axis_name}")
        ax.set_ylim(lim[0], lim[1])
        ax.set_xlim(lim[0], lim[1])

        corr_val = corr(y, [yhi.mean() for yhi in yhat])
        ax.text(lim[0] + 0.1 * (max(y) - min(y)), lim[1] - 1 * 0.1 * (max(y) - min(y)), f"(\u2191)correlation = {corr_val:.3f}")
        
        if GPR:
            ax.text(lim[0] + 0.1*(max(y)-min(y)), lim[1] - 2*0.1*(max(y)-min(y)), f"(\u2193)neg-ll = {log_likelihood(y, [yhi.mean() for yhi in yhat], yprobs, eps=1e-6):.3f}")
        else:
            ax.text(lim[0] + 0.1*(max(y)-min(y)), lim[1] - 2*0.1*(max(y)-min(y)), f"(\u2193)neg-ll = {log_likelihood(y, [yhi.mean() for yhi in yhat], [yhi.std() if len(yhi)>1 else max(yprobs) for yhi in yhat], eps=1e-6):.3f}")
        ax.text(lim[0] + 0.1*(max(y)-min(y)), lim[1] - 3*0.1*(max(y)-min(y)), f"(\u2193)MAE = {mae(y, [yhi.mean() for yhi in yhat]):.3f}")
        
        ax.plot(y,y)
        ax.plot(lim,lim)
        
        if GPR:
            ax.errorbar(y, 
                        [yhi.mean() for yhi in yhat], 
                        yerr=[abs(recal_bounds.lower),abs(recal_bounds.upper)],
                        fmt='.', color='gray', alpha=0.3)
        else:  
            ax.errorbar(y, 
                        [yhi.mean() for yhi in yhat], 
                        yerr=[yhi.std() if len(yhi)>1 else max(yprobs) for yhi in yhat],
                        fmt='.', color='gray', alpha=0.3)
        ax.scatter(
            y, [yhi.mean() for yhi in yhat], s=6, alpha=1, color=color
        )



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import uncertainty_toolbox as uct

#figsize=(6.4,4.8)
fig, axs = plt.subplots(nrows=3, ncols=7, figsize=(30,12), constrained_layout=True)
for ax in axs.flat:
    ax.set_aspect(1)


# plot axs[0,0]
d00 = select_df(df, data="C2", k=5, T=.7, model='text-curie-001', model_class='topk', N=1000)
lim_c2 = (min(d00['y']), max(d00['y']))
lim_c2 = (0, 1)
text_anchor = sum(lim_c2)/len(lim_c2)
create_sub_parity(axs[0,0], d00, axis_name='C2',model_class='topk', lim=lim_c2, color=f'C0',Type_cali="cali",rec_split=100)
# # plot axs[0,1]
d01 = select_df(df, data="C2", k=5, T=.7, model='text-curie-001', model_class='multi', N=1000)
create_sub_parity(axs[0,1], d01, axis_name='C2',model_class='multi', lim=lim_c2, color=f'C1', Type_cali="cali",rec_split=100)
# # plot axs[0,2]
d02 = select_df(df, data="C2", k=5, T=0.7, model='text-davinci-003', model_class='topk', N=1000)
create_sub_parity(axs[0,2], d02, axis_name='C2',model_class='topk', lim=lim_c2, color=f'C2', Type_cali="cali", rec_split=100,model='text-davinci-003')



# # plot axs[1,0]
d10 = select_df(df, data="C2", k=5, T=0.7, model='text-curie-001', model_class='topk', N=1000)
lim_c2 = (min(d10['y']), max(d10['y']))
lim_c2 = (0, 1)
create_sub_parity(axs[1,0], d10, axis_name='C2 yield',model_class='topk', lim=lim_c2, color=f'C4', Type_cali="recali",rec_split=100) # calibration plot
# # # plot axs[1,1]
d11 = select_df(df, data="C2", k=5, T=.7, model='text-curie-001', model_class='multi', N=1000)
lim = (min(d11['y']), max(d11['y']))
create_sub_parity(axs[1,1], d11, axis_name='C2 yield',model_class='multi', lim=lim_c2, color=f'C5', Type_cali="recali",rec_split=100)
# # plot axs[1,2]
d12 = select_df(df, data="C2", k=5, T=0.7, model='text-davinci-003', model_class='topk', N=1000)
lim = (min(d12['y']), max(d12['y']))
create_sub_parity(axs[1,2], d12, axis_name='C2 yield',model_class='topk', lim=lim_c2, color=f'C6',Type_cali="recali",rec_split=100)



# # plot axs[2,0]
d20 = select_df(df, data="C2", k=5, T=0.7, model='text-curie-001', model_class='topk', N=1000)
lim = (min(d20['y']), max(d20['y']))
create_sub_parity(axs[2,0], d20, 'C2 yield',model_class='topk', lim=lim_c2, color=f'C6',Type_cali="recali_scale",rec_split=100)

# # plot axs[2,1]
d21 = select_df(df, data="C2", k=5, T=0.7, model='text-curie-001', model_class='multi', N=1000)
lim = (min(d21['y']), max(d21['y']))
create_sub_parity(axs[2,1], d21, 'C2 yield',model_class='multi', lim=lim_c2, color=f'C6',Type_cali="recali_scale",GPR=False,rec_split=100)

# # # plot axs[2,2]
d22 = select_df(df, data="C2", k=5, T=0.7, model='text-davinci-003', model_class='topk', N=1000)
lim = (min(d22['y']), max(d22['y']))
create_sub_parity(axs[2,2], d22, 'C2 yield', lim=lim_c2, color=f'C6',Type_cali="recali_scale",rec_split=100)


# # plot axs[0,3]
d03 = select_df(df, data="C2", k=5, T=0.7, model='gpt-4', model_class='topk', N=1000)
lim_c2 = (min(d03['y']), max(d03['y']))
lim_c2 = (0, 1)
text_anchor = sum(lim_c2)/len(lim_c2)
lim = (min(d03['y']), max(d03['y']))
create_sub_parity(axs[0,3], d03, 'C2 yield',model='gpt-4', model_class='topk', lim=lim_c2, color=f'C6',Type_cali="cali",rec_split=100)

# # plot axs[1,3]
d33 = select_df(df, data="C2", k=5, T=0.7, model='gpt-4', model_class='topk', N=1000)
lim = (min(d33['y']), max(d33['y']))
create_sub_parity(axs[1,3], d33, 'C2 yield', lim=lim_c2, color=f'C6',Type_cali="recali",rec_split=100)

# # # plot axs[2,3]
d33 = select_df(df, data="C2", k=5, T=0.7, model='gpt-4', model_class='topk', N=1000)
lim = (min(d33['y']), max(d33['y']))
create_sub_parity(axs[2,3], d33, 'C2 yield', lim=lim_c2, color=f'C6',Type_cali="recali_scale",rec_split=100)




# # plot axs[0,4]
d04 = select_df(df, data="C2", k=0, T=0, model='GPR', model_class='GPR_mat', N=1000)
lim_c2 = (min(d04['y']), max(d04['y']))
lim_c2 = (0, 1)
text_anchor = sum(lim_c2)/len(lim_c2)
lim = (min(d04['y']), max(d04['y']))
create_sub_parity(axs[0,4], d04, 'C2 yield',model='GPR',GPR=True,model_class='GPR_mat', lim=lim_c2, color=f'C6',Type_cali="cali",rec_split=100)

# # plot axs[1,4]
d14 = select_df(df, data="C2", k=0, T=0, model='GPR', model_class='GPR_mat', N=1000)
lim = (min(d14['y']), max(d14['y']))
create_sub_parity(axs[1,4], d14, 'C2 yield', lim=lim_c2, color=f'C6',Type_cali="recali",GPR=True,rec_split=100)

# # # plot axs[2,4]
d24 = select_df(df, data="C2", k=0, T=0, model='GPR', model_class='GPR_mat', N=1000)
lim = (min(d24['y']), max(d24['y']))
create_sub_parity(axs[2,4], d24, 'C2 yield', lim=lim_c2, color=f'C6',Type_cali="recali_scale",GPR=True,rec_split=100)



# # plot axs[0,5]
d05 = select_df(df, data="C2", k=0, T=0, model='GPR', model_class='GPR_ada', N=1000)
lim_c2 = (min(d05['y']), max(d05['y']))
lim_c2 = (0, 1)
text_anchor = sum(lim_c2)/len(lim_c2)
lim = (min(d05['y']), max(d05['y']))
create_sub_parity(axs[0,5], d05, 'C2 yield',model='GPR',GPR=True, model_class='GPR-ada', lim=lim_c2, color=f'C6',Type_cali="cali",rec_split=100)

# # plot axs[1,5]
d35 = select_df(df, data="C2", k=0, T=0, model='GPR', model_class='GPR_ada', N=1000)
lim = (min(d35['y']), max(d35['y']))
create_sub_parity(axs[1,5], d35, 'C2 yield', lim=lim_c2, color=f'C6',Type_cali="recali",GPR=True,rec_split=100)

# # # plot axs[2,5]
d35 = select_df(df, data="C2", k=0, T=0, model='GPR', model_class='GPR_ada', N=1000)
lim = (min(d35['y']), max(d35['y']))
create_sub_parity(axs[2,5], d35, 'C2 yield', lim=lim_c2, color=f'C6',GPR=True,Type_cali="recali_scale",rec_split=100)



# # plot axs[0,6]
d06 = select_df(df, data="C2", k=0, T=0, model='GPR', model_class='GPR_num', N=1000)
lim_c2 = (min(d06['y']), max(d06['y']))
lim_c2 = (0, 1)
text_anchor = sum(lim_c2)/len(lim_c2)
lim = (min(d06['y']), max(d06['y']))
create_sub_parity(axs[0,6], d06, 'C2 yield',model='GPR',GPR=True,model_class='GPR_num', lim=lim_c2, color=f'C6',Type_cali="cali",rec_split=100)

# # plot axs[1,6]
d16 = select_df(df, data="C2", k=0, T=0, model='GPR', model_class='GPR_num', N=1000)
lim = (min(d16['y']), max(d16['y']))
create_sub_parity(axs[1,6], d16, 'C2 yield', lim=lim_c2, color=f'C6',Type_cali="recali",GPR=True,rec_split=100)

# # # plot axs[2,6]
d26 = select_df(df, data="C2", k=0, T=0, model='GPR', model_class='GPR_num', N=1000)
lim = (min(d26['y']), max(d26['y']))
create_sub_parity(axs[2,6], d26, 'C2 yield', lim=lim_c2, color=f'C6',Type_cali="recali_scale",GPR=True,rec_split=100)


plt.show()

In [ ]:
# import uncertainty_toolbox as uct

def combine(s, l):
  '''Number of combinations of l elements with max = s'''
  return (s**l - (s-1)**(l))

def prob(s, l, n):
  '''Probability of getting a sample with max([x0,x1,...,xl]) = s where xi={0,n}'''
  return combine(s,l) * ((1/n)**l)

def expected_value_p(l, n):
  '''Expected value of max([x0,x1,...,xl]) where xi={0,n}'''
  E = [s * prob(s, l, n) for s in range(1,100+1)]
  return sum(E)

def expected_value_q(l, n, data):
  '''Expected value of max([x0,x1,...,xl]) where xi={0,n}'''
  quants = [data.quantile(i/100) for i in range(100+1)]
  # E = [(quants[s-1]) * prob(s, l, n) for s in range(1,100+1)]
  E = [((quants[s-1]+quants[s])/2) * prob(s, l, n) for s in range(1,100+1)]
  return sum(E)

from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)

# @retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def run_experiment(
    asktell, pool, raw_data, indexes, x_name, y_name, N=1, initial_train=1, ask_K=1, aq="random", start_index=0, calibrate=False,
    lambda_multi=0.1, system_message="", inv_system_message="",transfer_train=1, transfer=False, trans_data=transfer_data, t_indexes=t_indexes
):
    if aq=='random_mean':
       return [ (i, expected_value_q(i, 100, raw_data[y_name])) for i in range(1,N+initial_train) ]
    
    point=[]
    py = 0 
    mod_std = 0
    counter = 0
    for i in indexes[:initial_train]:
        print("indexes check",)
        asktell.tell(raw_data[x_name].iloc[i], float(raw_data[y_name].iloc[i]))
        if counter == 0:
           i_best = float(raw_data[y_name].iloc[i])
        else:
           i_best = sorted(point, key=lambda i_points: i_points[-1])[-1][-3]
           
        point.append((raw_data[x_name].iloc[i],counter,i_best,float(raw_data[y_name].iloc[i]),float(raw_data[y_name].iloc[i]),mod_std))


        print("check",raw_data[x_name].iloc[i], float(raw_data[y_name].iloc[i]),i_best)
        counter+=1
        
    print("you made it")
    if transfer:
      for j in t_indexes[:transfer_train]:
          asktell.tell(trans_data[x_name].iloc[j], float(trans_data[y_name].iloc[j]))
          print("transfer running here and shouldn't")
        
    if calibrate:
        # y = [float(raw_data[y_name].iloc[i]) for i in indexes[:initial_train]]
        # pred = asktell.predict(y)
        # ymeans = np.array([yhi.mean() for yhi in pred])
        # ystds = np.array([yhi.std() for yhi in pred])
        # calibration_factor = uct.recalibration.optimize_recalibration_ratio (ymeans, ystds, np.array(y), criterion="miscal")
        calibration_factor = 5.0
        asktell.set_calibration_factor(calibration_factor)

    x = raw_data[x_name].tolist()

    pool.reset()
    xi = x[start_index]
    x.remove(xi)
    pool.choose(xi)
    print("len of x:",len(x),"startindex:",start_index,"xi:",xi)
    yi = float(raw_data[raw_data[x_name] == xi][y_name].iloc[0])
    asktell.tell(xi, yi) # Stuck here
    point.append((xi,1+initial_train,i_best, yi,py,mod_std))
    best = sorted(point, key=lambda points: points[-1])[-1][-3]


    for i in range(1, N):
        if i == N - 1 and aq != "random":
            aq = "greedy"
        px, _, py, mod_std = asktell.ask(pool,
                                k=ask_K,
                                aq_fxn=aq,
                                _lambda=1.0,
                                inv_filter=16,
                                aug_random_filter=0,
                                lambda_mult=lambda_multi,
                                system_message=system_message,
                                inv_system_message=inv_system_message,
                                )
        for j in range(ask_K):
          xc = px[j]
          x.remove(xc)
          pool.choose(xc)
          y = float(raw_data[raw_data[x_name] == xc][y_name].iloc[0])
          asktell.tell(xc, y)
          best = max(y, best)
          print("here",best)
        point.append((xc, 1+initial_train+i*ask_K, best, y,py[0],mod_std[0]))
        print(point)
    return point

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import urllib.request

urllib.request.urlretrieve(
    "https://github.com/google/fonts/raw/main/ofl/ibmplexmono/IBMPlexMono-Regular.ttf",
    "IBMPlexMono-Regular.ttf",
)
fe = font_manager.FontEntry(fname="IBMPlexMono-Regular.ttf", name="plexmono")
font_manager.fontManager.ttflist.append(fe)
plt.rcParams.update(
    {
        "axes.facecolor": "#f5f4e9",
        "grid.color": "#AAAAAA",
        "axes.edgecolor": "#333333",
        "figure.facecolor": "#FFFFFF",
        "axes.grid": False,
        "axes.prop_cycle": plt.cycler("color", plt.cm.Dark2.colors),
        "font.family": fe.name,
        "figure.figsize": (3.5, 3.5 / 1.2),
        "ytick.left": True,
        "xtick.bottom": True,
    }
)


In [ ]:
from scipy.stats import gaussian_kde
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df= pd.read_csv("out/dataset/data/C2_yield_meth_oxy_short_corrected.csv")

original_labels= df["completion"].values

kde = gaussian_kde(original_labels, bw_method="scott")
n_samples = len(df)
randomized_samples = kde.resample(n_samples).flatten()
randomized_samples = np.round(randomized_samples, 2)
np.random.shuffle(randomized_samples)
df['random_labels'] = randomized_samples 
df.to_csv("out/dataset/data/C2_yield_meth_oxy_short_corrected_wrandom_labels.csv", index=False)

kde_original = gaussian_kde(original_labels)
kde_random = gaussian_kde(randomized_samples)

x_min = min(original_labels.min(), randomized_samples.min())
x_max = max(original_labels.max(), randomized_samples.max())
x_vals = np.linspace(x_min, x_max, 1000)

plt.figure(figsize=(10, 6),)
plt.plot(x_vals, kde_original(x_vals), label='Original Labels KDE', linewidth=2)
plt.plot(x_vals, kde_random(x_vals), label='Randomized Labels KDE', linewidth=2, linestyle='--')
plt.title("KDE of Original vs. Randomized Labels",fontsize=14)
plt.xlabel("Label Value",fontsize=14)
plt.ylabel("Density",fontsize=14)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("out/figs/ocm_kde_original_vs_randomized.png", dpi=300)
plt.show()

